In [1]:
import pandas as pd
import json
fileName = '2021-06-10 10.17'
resPath = 'C:/Users/oliver.hurtig/Documents/Software/openLCA/UBA-results/'
with open(resPath + fileName + '.json') as f:
    data = json.load(f)
with open(resPath + fileName + '_units.json') as f:
    units = json.load(f)

C:\Users\oliver.hurtig\AppData\Local\Continuum\miniconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\oliver.hurtig\AppData\Local\Continuum\miniconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\oliver.hurtig\AppData\Local\Continuum\miniconda3\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
df = pd.DataFrame.from_dict({(system,co2,elec,impact): data[system][co2][elec]['categoryContrib'][impact]
                            for system in data.keys() 
                            for co2 in data[system].keys()
                            for elec in data[system][co2].keys()
                            for impact in data[system][co2][elec]['categoryContrib'].keys()},
                            orient='index')
df.index = pd.MultiIndex.from_tuples(df.index, names=('System', 'CO2', 'Strom', 'Wirkungskategorie'))

pd.set_option('display.max_rows', 80)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 2)
df.to_excel(resPath + fileName + '-full.xlsx', merge_cells=False, sheet_name='Categories')

df2 = pd.DataFrame.from_dict({(system,co2,elec,prcPath,impact): data[system][co2][elec]['processContrib'][prcPath][impact]
                            for system in data.keys() 
                            for co2 in data[system].keys()
                            for elec in data[system][co2].keys()
                            for prcPath in data[system][co2][elec]['processContrib'].keys()
                            for impact in data[system][co2][elec]['processContrib'][prcPath]},
                            orient='index')
df2.index = pd.MultiIndex.from_tuples(df2.index, names=('System', 'CO2', 'Strom', 'Prozess', 'Wirkungskategorie'))
with pd.ExcelWriter(resPath + fileName + '-full.xlsx', engine="openpyxl", mode='a') as writer:
    df2.to_excel(writer, merge_cells=False, sheet_name='Processes')

with pd.ExcelWriter(resPath + fileName + '.xlsx') as writer:
    for cat, new_df in df.groupby(level=3):
        new_df.to_excel(writer, merge_cells=False, sheet_name=cat[0:30])


In [175]:
#import matplotlib.pyplot as plt
import plotly.express as px
prod_systems = {
    'Syngas, Steam Methane Reforming': 'SMR',
    'synthesis gas (2:1), mixing of CO and H2 from PEM': 'PEM',
    'synthesis gas (2:1), mixing of CO and H2 from PYR': 'PYR'
}
def map_level(df, dct, level=0):
    index = df.index
    index.set_levels([[dct.get(item, item) for item in names] if i==level else names
                      for i, names in enumerate(index.levels)], inplace=True)

for cat, new_df in df.groupby(level=3):
    if cat == 'IPCC GWP 100a':
        map_level(new_df, prod_systems, level=0)
        ind = new_df.reset_index(level=3, drop=True).index.map('/'.join)
        new_df.index = ind
        new_df.index.name = 'Szenario'
        new_df.columns.name = 'Prozessschritt'
        fig = px.bar(new_df, orientation='h', color_discrete_sequence=px.colors.qualitative.Safe)
        fig.update_xaxes(title_text=cat + ' in ' + units[cat] + ' / kg Produkt')
        fig.update_layout(legend=dict(orientation='h', x=-0.2, y=-0.3))

fig.show()

ipykernel_launcher:11: FutureWarning:

inplace is deprecated and will be removed in a future version.



In [ ]:
!pip install plotly

In [79]:
#df.loc[(slice(None), slice(None), slice(None), 'IPCC GWP 100a')]

-KEA-Renewable, wind, solar, geothe    0.18        9.48e-05      6.97e-03              0                   4.34                 4.70                  2.65       
                                                  AZ  DE2017 z-KEA-Renewable, wind, solar, geothe    0.18        9.48e-05      2.03e-05              0                   1.38                 1.46                  1.09       
                                                      EE2050 z-KEA-Renewable, wind, solar, geothe    0.18        9.48e-05      2.03e-05              0                   4.34                 4.70                  3.50       
Syngas, Steam Methane Reforming                   DAC DE2017 z-KEA-Renewable, wind, solar, geothe    0.00        0.00e+00      0.00e+00              0                   0.00                 0.11                  0.00       
                                                      EE2050 z-KEA-Renewable, wind, solar, geothe    0.00        0.00e+00      0.00e+00              0                

In [30]:
df[df['Prozess'].endsWith('hydrogen, from electrolysis - EU-25')]

KeyError: 'Prozess'

In [2]:
!pip install xlwt

In [33]:
import pandas as pd
import json
from ipywidgets import Button
from tkinter import Tk, filedialog
from IPython.display import clear_output, display


res_path = 'C:/Users/oliver.hurtig/Documents/Software/openLCA/UBA-results/'
import traitlets
from ipywidgets import widgets
from IPython.display import display
from tkinter import Tk, filedialog

class SelectFilesButton(widgets.Button):
    """A file widget that leverages tkinter.filedialog."""

    def __init__(self):
        super(SelectFilesButton, self).__init__()
        # Add the selected_files trait
        self.add_traits(files=traitlets.traitlets.List())
        # Create the button.
        self.description = "Select Files"
        self.icon = "square-o"
        self.style.button_color = "orange"
        # Set on click behavior.
        self.on_click(self.select_files)

    @staticmethod
    def select_files(b):
        """Generate instance of tkinter.filedialog.

        Parameters
        ----------
        b : obj:
            An instance of ipywidgets.widgets.Button 
        """
        # Create Tk root
        root = Tk()
        # Hide the main window
        root.withdraw()
        # Raise the root to the top of all windows.
        root.call('wm', 'attributes', '.', '-topmost', True)
        # List of selected fileswill be set to b.value
        b.files = filedialog.askopenfilename(multiple=True, initialdir=res_path)

        b.description = "Files Selected"
        b.icon = "check-square-o"
        b.style.button_color = "lightgreen"

my_button = SelectFilesButton()
my_button


SelectFilesButton(description='Select Files', icon='square-o', style=ButtonStyle(button_color='orange'))

In [31]:
def writeExcel(fileName):
    with open(res_path + fileName + '.json') as f:
        data = json.load(f)
    system_sup = []
    co2_sup = []
    elec_sup = []
    impact_sup = []
    from xlwt import Workbook
    
    # Workbook is created
    wb = Workbook()
    writer = pd.ExcelWriter(res_path + fileName + '-systemwise.xlsx')
    sheet_count = 1
    for system in data:
        system_sup.append(system)
        for co2 in data[system]:
            if co2 not in co2_sup: 
                co2_sup.append(co2)
            for elec in data[system][co2]:
                if elec not in elec_sup:
                    elec_sup.append(elec)                  
                sheet_count += 1
                df = pd.DataFrame(data[system][co2][elec]['categoryContrib']).transpose().sort_index()
                df.to_excel(writer, sheet_name = '' + str(sheet_count) + system[0:3] + co2 + elec, index_label = system + '/' + co2 + '/' + elec)
    writer.save()
print(path)
print(my_button.files)
for path in my_button.files:
    fName = os.path.basename(path).split('.json')[0]
    print(fName)
    writeExcel(fName)

C:/Users/oliver.hurtig/Documents/Software/openLCA/UBA-results/2021-06-09 10.40.json
['C:/Users/oliver.hurtig/Documents/Software/openLCA/UBA-results/2021-06-09 10.40.json']
2021-06-09 10.40


In [42]:
from ipywidgets import interact


def select_co2(system_sel, co2_sel, elec_sel, impact_sel):
    print(data[system_sel][co2_sel][elec_sel]['categoryContrib'][impact_sel])

interact(select_co2, system_sel = system_sup, co2_sel = co2_sup, elec_sel = elec_sup, impact_sel = impact_sup)

,"synthesis gas (2:1), mixing of CO and H2 from PEM","Syngas, Steam Methane Reforming"
DAC,{'DE2017': {'processContrib': {'//synthesis ga...,"{'DE2017': {'processContrib': {'//Syngas, Stea..."
AZ,{'DE2017': {'processContrib': {'//synthesis ga...,"{'DE2017': {'processContrib': {'//Syngas, Stea..."


interactive(children=(Dropdown(description='system_sel', options=('synthesis gas (2:1), mixing of CO and H2 fr…

<function __main__.select_co2(system_sel, co2_sel, elec_sel, impact_sel)>

In [ ]:
dfn = pd.DataFrame.from_dict(data)
display(dfn)


In [ ]:
            for impact in data[system][co2][elec]['categoryContrib']:
                if impact not in impact_sup:
                    impact_sup.append(impact)
                sheet1.write(impact_count, 0, impact)
                impact_count += 1
                cat_count = 1
                for cat in data[system][co2][elec]['categoryContrib'][impact]:
                    if impact_count == 1:
                        sheet1.write(0, cat_count, cat)
                    sheet1.write(impact_count, cat_count, data[system][co2][elec]['categoryContrib'][impact][cat])
                    cat_count += 1

In [176]:
!pip install jupyter-dash

In [129]:
import plotly.graph_objects as go
trimmed = df2.query('System == "synthesis gas (2:1), mixing of CO and H2 from PEM" and CO2 == "DAC" and Strom == "DE2017" and Wirkungskategorie == "RMI abiotic"')
parents = []
labels = []
values = []
for i in trimmed.index.get_level_values(3):
    
    splitted = i.split('//')
    if len(splitted) < 5:
        labels.append(splitted[-1])
        parents.append(splitted[-2])
        values.append(trimmed.loc[(slice(None), slice(None), slice(None), i)].values[0][0])

values

[3.9418,
 0.0004,
 0.0,
 2.1475,
 0.0004,
 0.4281,
 0.6312,
 3.2069,
 0.0004,
 4.3827,
 7.5901,
 0.0002,
 0.4401]

In [130]:
go.Figure(go.Sunburst(
    labels=labels, parents=parents, values=values, branchvalues='total'
))

In [159]:
# Import libraries
from jupyter_dash import JupyterDash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
prod_systems = {
    'Syngas, Steam Methane Reforming': 'SMR',
    'synthesis gas (2:1), mixing of CO and H2 from PEM': 'PEM',
    'synthesis gas (2:1), mixing of CO and H2 from PYR': 'PYR'
}
def map_level(df, dct, level=0):
    index = df.index
    index.set_levels([[dct.get(item, item) for item in names] if i==level else names
                      for i, names in enumerate(index.levels)], inplace=True)
from zlib import crc32

def bytes_to_float(b):
    return float(crc32(b) & 0xffffffff) / 2**32
def str_to_float(s, encoding="utf-8"):
    return bytes_to_float(s.encode(encoding))
# Create the Dash app
app = JupyterDash()

# Set up the app layout
app.layout = html.Div(children=[
    html.H1(children='Prozessbeitrag Dashboard'),
    dcc.Dropdown(id='cat-dropdown',
                 options=[{'label': i, 'value': i}
                          for i in df.index.levels[3]],
                 value='IPCC GWP 100a'),
    dcc.Graph(id='contrib-graph'),
    dcc.Dropdown(id='sys-dropdown',
                 options=[{'label': i, 'value': i}
                          for i in df.index.levels[0]],
                 value=df.index.levels[0][0]),
    dcc.Dropdown(id='co2-dropdown',
                 options=[{'label': i, 'value': i}
                          for i in df.index.levels[1]],
                 value=df.index.levels[1][0]),
    dcc.Dropdown(id='strom-dropdown',
                 options=[{'label': i, 'value': i}
                          for i in df.index.levels[2]],
                 value=df.index.levels[2][0]),
    dcc.Graph(id='process-graph', figure={"layout": {"title": "My Dash Graph", "height": 700},},)
])

,
# Set up the callback function
@app.callback(
    Output(component_id='contrib-graph', component_property='figure'),
    Input(component_id='cat-dropdown', component_property='value')
)
def update_graph(selected_cat):
    for cat, new_df in df.groupby(level=3):
        if cat == selected_cat:
            filtered_df = new_df
    map_level(filtered_df, prod_systems, level=0)
    ind = filtered_df.reset_index(level=3, drop=True).index.map('/'.join)
    filtered_df.index = ind
    filtered_df.index.name = 'Szenario'
    filtered_df.columns.name = 'Prozessschritt'
    fig = px.bar(filtered_df, orientation='h', color_discrete_sequence=px.colors.qualitative.Safe)
    fig.update_xaxes(title_text=selected_cat + ' in ' + units[selected_cat] + ' / kg Produkt')
    fig.update_layout(legend=dict(orientation='h', x=-0.2, y=-0.3))
    return fig

@app.callback(
    Output(component_id='process-graph', component_property='figure'),
    [Input(component_id='cat-dropdown', component_property='value'), Input(component_id='sys-dropdown', component_property='value'), Input(component_id='co2-dropdown', component_property='value'), Input(component_id='strom-dropdown', component_property='value')]
)
def update_graph2(selected_cat, selected_sys, selected_co2, selected_strom):
    trimmed = df2.query('System == "'+ selected_sys +'" and CO2 == "'+selected_co2+'" and Strom == "'+selected_strom+'" and Wirkungskategorie == "' + selected_cat + '"')
    ids = []
    parents = []
    labels = []
    values = []
    for i in trimmed.index.get_level_values(3):        
        splitted = i.split('//')
        ids.append(i)
        labels.append(splitted[-1])
        parents.append(i.replace('//' + splitted[-1], ''))
        values.append(trimmed.loc[(slice(None), slice(None), slice(None), i)].values[0][0])
    colors = list(map(str_to_float, labels))
    fig = go.Figure(go.Sunburst(ids=ids, labels=labels, parents=parents, values=values, branchvalues='total', marker=dict(colors=colors, colorscale='RdBu')))    
    return fig


# Run local server
if __name__ == '__main__':
    app.run_server(mode='external')

Dash app running on http://127.0.0.1:8050/
ipykernel_launcher:18: FutureWarning:

inplace is deprecated and will be removed in a future version.



In [191]:
%tb

SystemExit: 1